In [ ]:
from crewai import Agent, Crew, Process, Task, LLM
from crewai.project import CrewBase, agent, crew, task
from crewai_tools import WebsiteSearchTool, ScrapeWebsiteTool, TXTSearchTool
from crewai.tools import BaseTool
from bs4 import BeautifulSoup
import requests
from crewai.tools import tool
import json

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ''
os.environ['SERPER_API_KEY'] = '' 

In [ ]:
from langchain.llms import Ollama
llm=LLM(model="ollama/llama3", base_url="http://localhost:11434")

In [ ]:
@tool("BrowserTool")
def scrape_and_summarize_website(source_url: str) -> str:
    """Useful to scrape and summarize a website content with financial news, blog, etc."""
    response = requests.get(source_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        relevant_elements = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'ul', 'ol', 'li', 'div', 'section', 'article'])    
        content = "\n\n".join([str(el) for el in relevant_elements])
        content_chunks = [content[i:i + 8000] for i in range(0, len(content), 8000)]
        summaries = []
        for chunk in content_chunks:
            agent = Agent(
                role='Principal Researcher',
                goal=
                'Do amazing research and summaries based on the content you are working with',
                backstory=
                "You're a Principal Researcher at a big company and you need to do research about a given topic.",
                allow_delegation=False,
                llm=llm)
            task = Task(
                agent=agent,
                description=
                f'Analyze and summarize the content below, make sure to include the most relevant information in the summary, return only the summary nothing else.\n\nCONTENT\n----------\n{chunk}',
                expected_output='A paragraph of the summary of the content provided. Should be detailed.'
            )
            summary = task.execute()
            summaries.append(summary)
        return "\n\n".join(summaries)
    else:
       return f"Failed to fetch content from {source_url}. Status code: {response.status_code}"

@tool("InternetSearch")
def search_internet(query: str) -> str:
    """Useful to search the internet 
    about a a given topic and return relevant results"""
    top_result_to_return = 4
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ['SERPER_API_KEY'],
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    results = response.json()['organic']
    string = []
    for result in results[:top_result_to_return]:
      try:
        string.append('\n'.join([
            f"Title: {result['title']}", f"Link: {result['link']}",
            f"Snippet: {result['snippet']}", "\n-----------------"
        ]))
      except KeyError:
        next

    return '\n'.join(string)

@tool("NewsSearch")
def search_news(query: str) -> str:
    """Useful to search news about a company, stock or any other
    topic and return relevant results"""""
    top_result_to_return = 4
    url = "https://google.serper.dev/news"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ['SERPER_API_KEY'],
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    results = response.json()['news']
    string = []
    for result in results[:top_result_to_return]:
      try:
        string.append('\n'.join([
            f"Title: {result['title']}", f"Link: {result['link']}",
            f"Snippet: {result['snippet']}", "\n-----------------"
        ]))
      except KeyError:
        next

    return '\n'.join(string)
  

In [ ]:
class ResearchAgents(): 
    def research_analyst(self): 
        return Agent(
            role = "The Best Research Assistant",
            goal = 'Impress all customers with your ability to gather and summarize information',
            backstory = "You are the BEST research analyst that is thorough and has a lot of experience gathering information and summarizing the most important parts for any topic given",
            verbose = True, 
            tools = [BrowserTools.scrape_and_summarize_website, 
                     SearchTools.search_news, 
                     SearchTools.search_internet, 
                     ScrapeWebsiteTool(), 
                     WebsiteSearchTool()], 
            llm = llm, 
            memory = True
        )

In [ ]:
research_assistant = Agent(
    role = 'Research Assistant', 
    goal = 'Gather all the information you can given a certain topic', 
    backstory = 'You are the BEST research assistant that is thorough and has a lot of experience gathering information regarding any topic that is given', 
    verbose = True, 
    tools = [scrape_and_summarize_website, 
             search_internet,
             search_news,
             ScrapeWebsiteTool(), 
             WebsiteSearchTool()], 
    llm = llm, 
    memory = True
)

research_task = Task(
    description = "Collect and summarize news articles that discuss the 2018-2022 US administration's stance on tariffs",
    expected_output = 'Return a chronological report that summarizes everything that the governement intends to do with regards to tariffs. Make sure you also highlight the parties that the tariffs will be levied against' ,
    agent = research_assistant,
    async_execution = True
)


In [ ]:
my_crew = Crew(
    agents = [research_assistant],
    tasks = [research_task],
    process = Process.sequential,
    verbose = True, 
)

In [ ]:
result = my_crew.kickoff()
print(result)